In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from datetime import datetime
from pathlib import Path
import pandas as pd

import torchtext.data as ttd

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [61]:
!pip install -q kaggle

In [4]:
from google.colab import files

In [5]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nabhsanjaymehtautd","key":"ee0e2e2e8b50d345f23e44404b090088"}'}

In [6]:
!mkdir ~/.kaggle/

In [7]:
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 60 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets list

ref                                                       title                                               size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
terenceshin/covid19s-impact-on-airport-traffic            COVID-19's Impact on Airport Traffic               106KB  2020-10-19 12:40:17           1644  
sootersaalu/amazon-top-50-bestselling-books-2009-2019     Amazon Top 50 Bestselling Books 2009 - 2019         15KB  2020-10-13 09:39:21           1602  
thomaskonstantin/highly-rated-children-books-and-stories  Highly Rated Children Books And Stories            106KB  2020-10-24 12:09:59            340  
tunguz/euro-parliament-proceedings-1996-2011              Euro Parliament Proceedings 1996 - 2011              1GB  2020-10-26 17:48:29             22  
rishidamarla/judicial-expenditures-across-all-50-states   Judicial Expenditures ac

In [10]:
!kaggle competitions download -c transferlearning-dl-spring2020

  0% 0.00/34.2k [00:00<?, ?B/s]
100% 34.2k/34.2k [00:00<00:00, 62.1MB/s]
  0% 0.00/506k [00:00<?, ?B/s]
100% 506k/506k [00:00<00:00, 71.6MB/s]
  0% 0.00/527k [00:00<?, ?B/s]
100% 527k/527k [00:00<00:00, 74.1MB/s]


In [11]:
!unzip train.csv.zip -d train

Archive:  train.csv.zip
  inflating: train/train.csv         


In [12]:
data = pd.read_csv('/content/train/train.csv', encoding = "ISO-8859-1")
testdata = pd.read_csv('/content/test.csv', encoding="ISO-8859-1")

In [13]:
data.head()

,id,text,target
0,86426,@USER She should ask a few native Americans wh...,1
1,16820,Amazon is investigating Chinese employees who ...,0
2,62688,"@USER Someone should'veTaken"" this piece of sh...",1
3,43605,@USER @USER Obama wanted liberals &amp; illega...,0
4,97670,@USER Liberals are all Kookoo !!!,1


In [14]:
data = data.drop(["id"], axis=1)

In [15]:
testdata = testdata.drop(["id"], axis = 1)

In [16]:
data.head()

,text,target
0,@USER She should ask a few native Americans wh...,1
1,Amazon is investigating Chinese employees who ...,0
2,"@USER Someone should'veTaken"" this piece of sh...",1
3,@USER @USER Obama wanted liberals &amp; illega...,0
4,@USER Liberals are all Kookoo !!!,1


In [17]:
data['target'].value_counts()

0    6220
1    3126
Name: target, dtype: int64

In [18]:
data.columns = ['data','labels']

In [19]:
testdata.columns = ['data']

In [20]:
testdata.head()

,data
0,@USER @USER Go home youâre drunk!!! @USER #M...
1,@USER @USER Oh noes! Tough shit.
2,@USER Canada doesnât need another CUCK! We a...
3,@USER @USER @USER It should scare every Americ...
4,@USER @USER @USER @USER LOL!!! Throwing the ...


In [21]:
data.head()

,data,labels
0,@USER She should ask a few native Americans wh...,1
1,Amazon is investigating Chinese employees who ...,0
2,"@USER Someone should'veTaken"" this piece of sh...",1
3,@USER @USER Obama wanted liberals &amp; illega...,0
4,@USER Liberals are all Kookoo !!!,1


In [22]:
testdata = pd.Series(testdata['data'].to_numpy())

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['labels'], test_size=0.3)

In [24]:
type(X_test)

pandas.core.series.Series

In [25]:
print(X_train.shape,y_train.shape)

(6542,) (6542,)


In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_tok = tokenizer.texts_to_sequences(X_train)
X_test_tok = tokenizer.texts_to_sequences(X_test)

In [27]:
testdata_tok = tokenizer.texts_to_sequences(testdata)

In [28]:
print(len(testdata_tok))

3894


In [29]:
print(len(X_train_tok)) # it is a list , does not have shape attribute

6542


In [30]:
# get words in each review
train_review_lengths = [len(x) for x in X_train_tok]
test_teview_lengths = [len(x) for x in X_test_tok]

testdata_review_lenghts = [len(x) for x in testdata_tok]

In [31]:
# get word -> integer mapping
word2idx = tokenizer.word_index
V = len(word2idx)
print(f'Unique Tokens {V}')

Unique Tokens 14271


In [32]:
# pad sequences so that we get a N x T matrix

# This function transforms a list (of length num_samples) of sequences (lists of integers).
# into a 2D Numpy array of shape (num_samples, num_timesteps).
# num_timesteps is either the maxlen argument if provided,
# or the length of the longest sequence in the list.

X_train_tok_pad = pad_sequences(X_train_tok) 
print(f'Shape of X_train_tok_pad : {X_train_tok_pad.shape}')


Shape of X_train_tok_pad : (6542, 102)


In [33]:
testdata_tok_pad = pad_sequences(testdata_tok)

In [34]:
# get sequence length
T = X_train_tok_pad.shape[1]
T

102

In [35]:
Test_T = testdata_tok_pad.shape[1]

In [36]:
Test_T

98

In [37]:
X_test_tok_pad = pad_sequences(X_test_tok, maxlen=T)
print(f'Shape of X_test_tok_pad : {X_test_tok_pad.shape}')

Shape of X_test_tok_pad : (2804, 102)


In [38]:
X_train_tensor = torch.from_numpy(X_train_tok_pad).long() #int64
y_train_tensor = torch.from_numpy(y_train.to_numpy()) # pandas series to numpy , numpy to tensor
X_test_tensor = torch.from_numpy(X_test_tok_pad).long() # int64
y_test_tensor = torch.from_numpy(y_test.to_numpy())

In [39]:
testdata_tensor = torch.from_numpy(testdata_tok_pad).long()

In [40]:
# create dataset objects
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor,y_test_tensor)

In [41]:
testdata_dataset = torch.utils.data.TensorDataset(testdata_tensor)

In [42]:
# Data loaders
train_iter = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=32, 
                                           shuffle=True)

test_iter = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=256, 
                                          shuffle=False)

In [43]:
testdata_iter = torch.utils.data.DataLoader(dataset=testdata_dataset,
                                            batch_size = 64,
                                            shuffle = False)

In [44]:
for i in testdata_iter:
  print(i)
  break

[tensor([[    0,     0,     0,  ...,    38,  1171,    16],
        [    0,     0,     0,  ...,   212,  1676,    89],
        [    0,     0,     0,  ...,   153,   131, 12513],
        ...,
        [    0,     0,     0,  ...,     5, 12609,  1845],
        [    0,     0,     0,  ...,  1468,    53,   814],
        [    0,     0,     0,  ..., 11248,    39,    16]])]


In [45]:
for inputs, targets in train_iter:
  print("inputs:", inputs, "shape:", inputs.shape)
  print("targets:", targets, "shape:", targets.shape)
  break

inputs: tensor([[   0,    0,    0,  ...,   97, 1017,  260],
        [   0,    0,    0,  ...,  336,  143, 1673],
        [   0,    0,    0,  ...,   54,  238,  372],
        ...,
        [   0,    0,    0,  ...,  305,   17,  127],
        [   0,    0,    0,  ...,    5,  153,  919],
        [   0,    0,    0,  ...,   38, 4739,   16]]) shape: torch.Size([32, 102])
targets: tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 1, 0, 0, 0, 0]) shape: torch.Size([32])


In [46]:
for inputs, targets in test_iter:
  print("inputs:", inputs)
  print("targets:", targets)
  break

inputs: tensor([[   0,    0,    0,  ..., 6087,   91,  752],
        [   0,    0,    0,  ...,   70,   61,  857],
        [   0,    0,    0,  ...,  527,   27,  333],
        ...,
        [   0,    0,    0,  ...,    2, 2787, 1392],
        [   0,    0,    0,  ...,  934,   13,  278],
        [   0,    0,    0,  ...,    6,  275, 7291]])
targets: tensor([0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
        1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
        1, 0, 0, 

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [48]:
# Define the model
class RNN(nn.Module):
  def __init__(self, n_vocab, embed_dim, n_hidden, n_rnnlayers, n_outputs):
    super(RNN, self).__init__()
    self.V = n_vocab
    self.D = embed_dim
    self.M = n_hidden
    self.K = n_outputs
    self.L = n_rnnlayers

    self.embed = nn.Embedding(self.V, self.D)
    self.rnn = nn.LSTM(
        input_size=self.D,
        hidden_size=self.M,
        num_layers=self.L,
        batch_first=True)
    self.fc = nn.Linear(self.M, self.K)
  
  def forward(self, X):
    # initial hidden states
    h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
    c0 = torch.zeros(self.L, X.size(0), self.M).to(device)

    # embedding layer
    # turns word indexes into word vectors
    out = self.embed(X)

    # get RNN unit output
    out, _ = self.rnn(out, (h0, c0))

    # max pool
    out, _ = torch.max(out, 1)

    # we only want h(T) at the final time step
    out = self.fc(out)
    return out

In [49]:
model = RNN(V+1, 300, 100, 1, 3) # V= len(vocab) + 1 for (padding)
model.to(device)

RNN(
  (embed): Embedding(14272, 300)
  (rnn): LSTM(300, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)

In [50]:
print(model)

RNN(
  (embed): Embedding(14272, 300)
  (rnn): LSTM(300, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)


In [51]:
for name, param in model.named_parameters():
  print(name, param.shape)

embed.weight torch.Size([14272, 300])
rnn.weight_ih_l0 torch.Size([400, 300])
rnn.weight_hh_l0 torch.Size([400, 100])
rnn.bias_ih_l0 torch.Size([400])
rnn.bias_hh_l0 torch.Size([400])
fc.weight torch.Size([3, 100])
fc.bias torch.Size([3])


In [52]:
learning_rate = 0.01
epochs=65
# STEP 5: INSTANTIATE LOSS CLASS
criterion = nn.CrossEntropyLoss()

# STEP 6: INSTANTIATE OPTIMIZER CLASS

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# STEP 7: TRAIN THE MODEL

train_losses= np.zeros(epochs)
test_losses= np.zeros(epochs)


for epoch in range(epochs):
  
  t0= datetime.now()
  train_loss=[]
  
  model.train()
  for input,targets in train_iter:
    # load input and output to GPU
    input = input.to(device)
    targets= targets.to(device)
    
    # forward pass
    output= model(input)
    loss=criterion(output,targets)

    # set gradients to zero 
    optimizer.zero_grad()

    # backward pass
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
  
  train_loss=np.mean(train_loss)
      
  test_loss=[]
  model.eval()
  with torch.no_grad():
    for input,targets in test_iter:
      # load input and output to GPU
      input = input.to(device)
      targets= targets.to(device)
      
  
      # forward pass
      output= model(input)
      loss=criterion(output,targets)
      test_loss.append(loss.item())

    test_loss=np.mean(test_loss)
  
  # save Losses
  train_losses[epoch]= train_loss
  test_losses[epoch]= test_loss
  dt= datetime.now()-t0
  print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}    Test Loss: {test_loss:.4f}, Duration: {dt}')

Epoch 1/65, Train Loss: 0.7063    Test Loss: 0.6634, Duration: 0:00:01.490683
Epoch 2/65, Train Loss: 0.6409    Test Loss: 0.6506, Duration: 0:00:01.279411
Epoch 3/65, Train Loss: 0.6327    Test Loss: 0.6427, Duration: 0:00:01.264833
Epoch 4/65, Train Loss: 0.6277    Test Loss: 0.6407, Duration: 0:00:01.277945
Epoch 5/65, Train Loss: 0.6247    Test Loss: 0.6371, Duration: 0:00:01.263438
Epoch 6/65, Train Loss: 0.6206    Test Loss: 0.6351, Duration: 0:00:01.262783
Epoch 7/65, Train Loss: 0.6179    Test Loss: 0.6330, Duration: 0:00:01.276571
Epoch 8/65, Train Loss: 0.6139    Test Loss: 0.6349, Duration: 0:00:01.376868
Epoch 9/65, Train Loss: 0.6096    Test Loss: 0.6329, Duration: 0:00:01.273635
Epoch 10/65, Train Loss: 0.6049    Test Loss: 0.6292, Duration: 0:00:01.292015
Epoch 11/65, Train Loss: 0.6004    Test Loss: 0.6242, Duration: 0:00:01.270034
Epoch 12/65, Train Loss: 0.5956    Test Loss: 0.6234, Duration: 0:00:01.270217
Epoch 13/65, Train Loss: 0.5905    Test Loss: 0.6188, Duratio

In [53]:
# Accuracy- write a function to get accuracy
# use this function to get train/test accuracy and print accuracy
def get_accuracy(train_iter, test_iter, model):
  model.eval()
  with torch.no_grad():
    correct_train=correct_test=0
    total_train=total_test=0
    
    for input, targets in train_iter:
      input= input.to(device)
      targets= targets.to(device)
      #input = input.view(-1, 784)
      output=model(input)
      _,indices = torch.max(output,dim=1)
      correct_train+= (targets==indices).sum().item()
      total_train += targets.shape[0]
    
    train_acc= correct_train/total_train

    for input, targets in test_iter:
      input= input.to(device)
      targets= targets.to(device)
      #input = input.view(-1, 784)
      output=model(input)
      _,indices = torch.max(output,dim=1)
      correct_test+= (targets==indices).sum().item()
      total_test += targets.shape[0]
    
    test_acc= correct_test/total_test
    return train_acc, test_acc

In [54]:
train_acc, test_acc = get_accuracy(train_iter, test_iter, model)
print(f'Train acc: {train_acc:.4f},\t Test acc: {test_acc:.4f}')

Train acc: 0.9633,	 Test acc: 0.7204


In [59]:
# Write a function to get predictions

def get_predictions(test_iter, model):
  model.eval()
  with torch.no_grad():
    predictions= np.array([])
    y_test= np.array([])

    for batch in test_iter:
      
      output=model(batch.data)
      _,indices = torch.max(output,dim=1)
      predictions=np.concatenate((predictions,indices.cpu().numpy())) 
      y_test = np.concatenate((y_test,batch.label.cpu().numpy())) 

  return y_test, predictions

In [60]:
predictions = get_predictions(testdata_iter, model)

AttributeError: ignored

In [ ]:
y_test, predictions=get_predictions(test_iter, model)

In [ ]:
predictions.max()

In [ ]:
# We are using confusion metrics from sklearn
# we are done with model building and predictions
# let us convert test data set to numpy arrays now

#y_test=y_test.numpy()

cm=confusion_matrix(y_test,predictions)
cm

In [ ]:
# Write a function to print confusion matrix
# plot confusion matrix
# need to import confusion_matrix from sklearn for this function to work
# need to import seaborn as sns
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true,y_pred,normalize=None):
  cm=confusion_matrix(y_true,y_pred,normalize=normalize)
  fig, ax = plt.subplots(figsize=(6,5))
  if normalize == None:
    fmt='d'
    fig.suptitle('Confusion matrix without Normalization', fontsize=12)
        
  else :
    fmt='0.2f'
    fig.suptitle('Normalized confusion matrix', fontsize=12)
    
  ax=sns.heatmap(cm,cmap=plt.cm.Blues,annot=True,fmt=fmt)
  ax.axhline(y=0, color='k',linewidth=1)
  ax.axhline(y=cm.shape[1], color='k',linewidth=2)
  ax.axvline(x=0, color='k',linewidth=1)
  ax.axvline(x=cm.shape[0], color='k',linewidth=2)
 
  ax.set_xlabel('Predicted label', fontsize=12)
  ax.set_ylabel('True label', fontsize=12)

In [ ]:
plot_confusion_matrix(y_test,predictions)

In [ ]:
predictions.shape

In [ ]:
test_data = pd.read_csv('/content/test.csv', encoding = "ISO-8859-1")

In [ ]:
data.shape

In [ ]:
pd.DataFrame({'Id': test_data.id[:2804], 'target': predictions}).to_csv('solution_base.csv', index =False)

In [ ]:
pwd